In [1]:
import numpy as np

In [229]:
def reconstruct(groups, group_sizes, strong_set, strong_order, strong_begins, strong_beta, indices, vals):
    out = np.copy(strong_beta)
    outer_pos = 0
    inner_pos = 0
    so = strong_order[outer_pos]
    group = groups[strong_set[so]]
    group_size = group_sizes[strong_set[so]]

    for i, v in zip(indices, vals):
        while outer_pos < len(strong_set):
            out[strong_begins[so] + inner_pos : strong_begins[so] + min(group_size, i - group)] = 0

            if i < group + group_size:
                inner_pos = i - group
                out[strong_begins[so] + inner_pos] = v
                inner_pos += 1
                break

            inner_pos = 0
            outer_pos += 1
            if outer_pos < len(strong_set):
                so = strong_order[outer_pos]
                group = groups[strong_set[so]]
                group_size = group_sizes[strong_set[so]]

    if outer_pos == len(strong_set):
        return out

    out[(strong_begins[so] + inner_pos) : (strong_begins[so] + group_size)] = 0
    outer_pos += 1
        
    # zero out the rest
    while outer_pos < len(strong_set):
        out[strong_begins[so] : (strong_begins[so] + group_size)] = 0
        outer_pos += 1
        if outer_pos < len(strong_set):
            so = strong_order[outer_pos]
            group_size = group_sizes[strong_set[so]]
        
    return out

In [230]:
def checker(p, indices, vals, strong_set, groups, group_sizes):
    out = np.zeros(p)
    for i, v in zip(indices, vals):
        out[i] = v
    
    strong_beta = np.concatenate([
        out[groups[i] : (groups[i] + group_sizes[i])]
        for i in strong_set
    ])
    return strong_beta, out

In [234]:
np.random.seed(0)

groups = np.array([0, 2, 5, 10, 21, 25, 30, 40, 50, 80], dtype=int)
strong_set = np.array([3, 2, 0, 5, 7, 8], dtype=int)

group_sizes = (groups[1:] - groups[:-1])
groups = groups[:-1]
p = np.sum(group_sizes, dtype=int)
strong_order = np.argsort(strong_set)
strong_begins = np.cumsum([0] + [group_sizes[i] for i in strong_set])[:-1]
strong_beta = np.concatenate(
    [
        np.random.normal(0, 1, (group_sizes[i],))
        for i in strong_set
    ]
)

indices = np.sort(np.concatenate([np.arange(groups[s], groups[s] + group_sizes[s]) for s in strong_set]))
indices_sub = np.sort(np.random.choice(indices, size=7, replace=False))
vals_sub = np.array(np.random.normal(0, 1, len(indices_sub)), dtype=int)
expected = checker(p, indices_sub, vals_sub, strong_set, groups, group_sizes)[0]

In [235]:
vals_sub

array([-1, -1,  0,  1, -1, -2, -2])

In [236]:
actual = reconstruct(
    groups, group_sizes, strong_set, strong_order, strong_begins, strong_beta, indices_sub, vals_sub
)
np.all(actual == expected), expected, actual

(True,
 array([-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
         0.,  0.,  0.,  0., -2.,  0.,  0.,  0., -2.,  0.,  0.]),
 array([-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
         0.,  0.,  0.,  0., -2.,  0.,  0.,  0., -2.,  0.,  0.]))